# 🍏 Lesion-Aware Explainability Validation for Apple Disease CNN
### This notebook implements a pipeline to validate whether a CNN focuses on true lesion areas (via Grad-CAM) when classifying apple leaf diseases. We compare the generated heatmaps with ground-truth lesion masks using Intersection over Union (IoU).


## 📦 1. Install Required Libraries
## Ensure required packages are available.

In [ ]:
!pip install tensorflow matplotlib opencv-python pandas numpy --quiet

## 📚 2. Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
import cv2 # OpenCV for image manipulation
import os
import pandas as pd

In [ ]:
def set_seed(seed=42):
    """
    Sets the seed for reproducibility across Python, NumPy, and TensorFlow.

    Args:
        seed (int): The seed value to use for all random generators. Default is 42.
    """
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

## ⚙️ 3. Global Configuration
### Define constants for image dimensions and model layers.

In [ ]:
# --- Configuration ---
IMG_WIDTH, IMG_HEIGHT = 224, 224 # Standard input size for ResNet50
LAST_CONV_LAYER_NAME = "conv5_block3_out" # Last convolutional layer in ResNet50
CLASSIFIER_LAYER_NAME = "predictions" # Final classification layer in ResNet50

## 🖼️ 4. Data Loading & Preprocessing Functions

In [ ]:
def load_and_preprocess_image(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT)):
    """
    Loads an image from disk and preprocesses it for ResNet50 input.

    - Resizes the image to the target size.
    - Converts it to a NumPy array and expands dimensions.
    - Applies ResNet50's preprocessing (scaling, normalization).

    Args:
        img_path (str): Path to the input image.
        target_size (tuple): Desired image size as (width, height).

    Returns:
        tuple:
            - Preprocessed image ready for model input (shape: (1, H, W, C)).
            - Original unprocessed image array (for visualization).
    """
    img = keras_image.load_img(img_path, target_size=target_size)
    img_array = keras_image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded), img_array # Return original for display

def load_segmentation_mask(mask_path, target_size=(IMG_WIDTH, IMG_HEIGHT)):
    """
    Loads a binary segmentation mask from disk and ensures it is properly resized and thresholded.

    - Loads the mask in grayscale mode.
    - Normalizes pixel values to [0, 1] range.
    - Thresholds to produce a binary mask (0 or 1).

    Assumes lesion pixels have intensity > 0 after normalization.

    Args:
        mask_path (str): Path to the ground-truth lesion mask.
        target_size (tuple): Desired mask size as (width, height).

    Returns:
        np.array: Binary segmentation mask with values 0 (background) and 1 (lesion).
    """
    mask = keras_image.load_img(mask_path, target_size=target_size, color_mode="grayscale")
    mask_array = keras_image.img_to_array(mask)
    mask_array = mask_array / 255.0 # Normalize to 0-1
    # Ensure mask is binary (0 or 1) - threshold if necessary
    # This depends on your mask format. If it's already 0/1, this might not be needed.
    # If masks have values like 0 and 255, the division by 255 handles it.
    # If masks have other values for lesions, adjust thresholding.
    # For this example, we assume masks are single channel, with lesion pixels > 0.
    mask_array_binary = (mask_array > 0.5).astype(np.uint8) # Threshold to make it strictly binary
    return mask_array_binary

#### To generate ground-truth lesion masks for the real images

In [ ]:
def generate_lesion_mask_from_image(original_img_array):
    """
    Generates a binary lesion mask from an RGB image using simple HSV thresholding.

    This is a heuristic placeholder method for cases where ground-truth masks are missing.
    It converts the RGB image to HSV color space and applies predefined threshold ranges
    to isolate potential lesion regions. The mask is returned as a binary array.

    Note:
        - The HSV threshold values used here are dataset-dependent and may need tuning.
        - This approach is approximate and may not capture lesions accurately.

    Args:
        original_img_array (np.array): The original RGB image as a NumPy array.

    Returns:
        np.array: Binary lesion mask with 1 indicating lesion regions, 0 otherwise.
    """
    hsv_img = cv2.cvtColor(original_img_array.astype(np.uint8), cv2.COLOR_RGB2HSV)

    # Example threshold — tweak these based on dataset inspection
    lower = np.array([10, 30, 30])
    upper = np.array([60, 255, 255])

    lesion_mask = cv2.inRange(hsv_img, lower, upper)
    binary_mask = (lesion_mask > 0).astype(np.uint8)  # 1 where lesion, 0 elsewhere
    return binary_mask

## 🧠 5. Load Pretrained ResNet50 Model

In [ ]:
# --- 2. Classification Model ---
def get_classification_model():
    """
    Loads a pre-trained ResNet50 model for image classification.

    - The model is loaded with ImageNet weights and includes the top classification layers.
    - Used as a baseline for apple leaf disease classification.
    - Can optionally be fine-tuned on a task-specific dataset (not done here).

    Returns:
        tf.keras.Model: A ResNet50 model ready for inference or fine-tuning.
    """
    base_model = ResNet50(weights='imagenet', include_top=True)
    # For a real application, you would fine-tune this model on your apple disease dataset.
    # For this example, we use it as is.
    return base_model

## 🔍 6. Grad-CAM Heatmap Generation

In [ ]:
def make_gradcam_heatmap(img_array_preprocessed, model, last_conv_layer_name, classifier_layer_name, pred_index=None):
    """
    Generates a Grad-CAM heatmap for a given input image and CNN model.

    Grad-CAM (Gradient-weighted Class Activation Mapping) highlights the regions in
    the input image that the model considers important for predicting a specific class.

    Args:
        img_array_preprocessed (np.array): Preprocessed input image array of shape (1, H, W, C).
        model (tf.keras.Model): Trained CNN model.
        last_conv_layer_name (str): Name of the last convolutional layer in the model.
        classifier_layer_name (str): Name of the classifier layer (not used here, included for API compatibility).
        pred_index (int, optional): Target class index. If None, uses the top predicted class.

    Returns:
        tuple:
            - heatmap (np.array): A 2D array with values normalized between 0 and 1.
            - pred_index (int): Index of the class used to generate the heatmap.
    """
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array_preprocessed)

        if pred_index is None:
            pred_index = tf.argmax(preds[0])
            pred_index = int(pred_index.numpy())  # ✅ Ensure scalar integer

        class_channel = preds[:, pred_index]
        grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy(), pred_index

## 🎯 7. Threshold Heatmap to Binary Mask

In [ ]:
def binarize_heatmap(heatmap, threshold=0.5):
     """
    Converts a continuous heatmap into a binary mask using a fixed threshold.

    This function sets all values greater than the specified threshold to 1 (lesion)
    and the rest to 0 (background), producing a binary mask for evaluation or visualization.

    Args:
        heatmap (np.array): The input heatmap array with values typically in [0, 1].
        threshold (float): Threshold value to binarize the heatmap. Default is 0.5.

    Returns:
        np.array: Binary mask of the same shape as the heatmap, with values 0 or 1.
    """
    return (heatmap > threshold).astype(np.uint8)

## 📏 8. IoU Computation

In [ ]:
def calculate_iou(mask1, mask2):
    """
    Calculates the Intersection over Union (IoU) score between two binary masks.

    IoU is a common metric used to evaluate the overlap between two segmentation masks,
    defined as the ratio of the intersection area to the union area.

    Args:
        mask1 (np.array): First binary mask (values 0 or 1).
        mask2 (np.array): Second binary mask (values 0 or 1).

    Returns:
        float: IoU score (value between 0 and 1). Higher is better.
    """
    # Ensure masks are boolean or 0/1 integer type
    mask1 = mask1.astype(bool)
    mask2 = mask2.astype(bool)
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

## 🔁 9. Full Pipeline for One Image
### Includes classification, heatmap generation, mask comparison, IoU calculation, and visualization.

In [ ]:
def process_image_and_validate_focus(image_path, classification_model, mask_dir, predicted_dir, return_visuals=False):
    """
    Processes a single image to assess whether the CNN's Grad-CAM attention aligns with actual lesion regions.

    This function:
    - Loads and preprocesses the input image.
    - Predicts the class using the classification model.
    - Generates the Grad-CAM heatmap and binarizes it.
    - Loads or generates the corresponding ground-truth lesion mask.
    - Computes the Intersection over Union (IoU) score between Grad-CAM and the lesion mask.
    - Optionally visualizes or returns results for external saving.

    Args:
        image_path (str): Path to the input image.
        classification_model (tf.keras.Model): Trained CNN model (e.g., ResNet50).
        mask_dir (str): Directory containing ground-truth lesion masks.
        predicted_dir (str): Directory to save generated Grad-CAM binary masks.
        return_visuals (bool, optional): If True, returns intermediate visuals instead of plotting inline.

    Returns:
        If return_visuals is False:
            tuple: (base_name, iou_score) where:
                - base_name (str): Name of the processed image (without extension).
                - iou_score (float): Computed Intersection over Union score.

        If return_visuals is True:
            tuple: (base_name, iou_score, original_img_array, true_segmentation_mask,
                    heatmap_resized, binarized_grad_cam_mask, predicted_class_name)
    """
    base_name, _ = os.path.splitext(os.path.basename(image_path))
    image_class = base_name.split('_')[0]

    # Load image and preprocess
    preprocessed_img, original_img_array = load_and_preprocess_image(image_path)

    # Prediction
    predictions = classification_model.predict(preprocessed_img)
    decoded_preds = decode_predictions(predictions, top=3)
    top_pred_index = np.argmax(predictions)

    # Grad-CAM
    heatmap, _ = make_gradcam_heatmap(
        preprocessed_img, classification_model,
        LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAME,
        pred_index=top_pred_index
    )
    heatmap_resized = cv2.resize(heatmap, (IMG_WIDTH, IMG_HEIGHT))
    binarized_grad_cam_mask = binarize_heatmap(heatmap_resized, threshold=0.5)

    # Save predicted Grad-CAM mask
    predicted_mask_filename = f"{base_name}_gradcam.jpg"
    predicted_mask_path = os.path.join(predicted_dir, predicted_mask_filename)
    keras_image.save_img(predicted_mask_path, np.expand_dims(binarized_grad_cam_mask * 255, axis=-1), scale=False)

    # Load real ground truth mask
    true_mask_filename = f"{base_name}_masked.jpg"
    true_mask_path = os.path.join(mask_dir, true_mask_filename)
    if not os.path.exists(true_mask_path):
        print(f"🛠️ Ground truth lesion mask not found. Generating: {true_mask_filename}")
        lesion_mask = generate_lesion_mask_from_image(original_img_array)
        keras_image.save_img(true_mask_path, np.expand_dims(lesion_mask * 255, axis=-1), scale=False)

    true_segmentation_mask = load_segmentation_mask(true_mask_path)

    # Resize GT mask if needed
    if true_segmentation_mask.shape != (IMG_HEIGHT, IMG_WIDTH):
        true_segmentation_mask = cv2.resize(true_segmentation_mask, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)

    # Calculate IoU
    iou_score = calculate_iou(binarized_grad_cam_mask, true_segmentation_mask)
    print(f"IoU for {base_name}: {iou_score:.4f}")

    if return_visuals:
        # Return all visuals for saving later
        return (
            base_name,
            iou_score,
            original_img_array,
            true_segmentation_mask,
            heatmap_resized,
            binarized_grad_cam_mask,
            decoded_preds[0][1]  # Top predicted class name
        )
    else:
        # Default behavior: visualize inline
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))
        axs[0].imshow(original_img_array / 255.0)
        axs[0].set_title(f"Original\nPred: {decoded_preds[0][1]}")

        axs[1].imshow(true_segmentation_mask, cmap='gray')
        axs[1].set_title("Ground Truth Mask")

        axs[2].imshow(original_img_array / 255.0)
        axs[2].imshow(heatmap_resized, cmap='jet', alpha=0.5)
        axs[2].set_title("Grad-CAM")

        axs[3].imshow(binarized_grad_cam_mask, cmap='gray')
        axs[3].set_title(f"Binary CAM\nIoU: {iou_score:.2f}")
        for ax in axs: ax.axis('off')
        plt.tight_layout()
        plt.show()

        return base_name, iou_score

## 🧪 10. Run for all images in the dataset

In [ ]:
import shutil

shutil.rmtree("/content/apple_plant_dataset", ignore_errors=True)
shutil.rmtree("/content/dataset.zip", ignore_errors=True)

In [ ]:
# ✅ Upload, unzip, and set dataset paths for 3 folders
from google.colab import files
uploaded = files.upload()

import zipfile, os

# Define extraction path
zip_file_path = list(uploaded.keys())[0]
extract_dir = "./apple_plant_dataset"

# Unzip to the extraction directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# 🗂️ Define paths
dataset_root = extract_dir
image_dir = os.path.join(dataset_root, "images")
mask_dir = os.path.join(dataset_root, "masks")
predicted_dir = os.path.join(dataset_root, "predicted")
results_dir = "results"  # outside apple_plant_dataset

Saving dataset.zip to dataset.zip


#### Using the model, all 60 images uploaded will be used for visualizations generation

In [ ]:
import csv

# --- Create folders if missing ---
os.makedirs(image_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)
os.makedirs(predicted_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# --- Create subfolder for visualizations ---
visualization_dir = os.path.join(results_dir, "initial_visualizations")
os.makedirs(visualization_dir, exist_ok=True)

# --- Load model ---
classification_model = get_classification_model()

# --- Class label mapping (for fine-tuning later) ---
class_mapping = {"healthy": 0, "rust": 1, "scab": 2}

# --- IoU results ---
iou_results = [("image_name", "iou_score")]

# --- Loop through real images ---
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_dir, filename)

        # Run your modified focus function and get back extra outputs
        base_name, iou_score, original_img, gt_mask, heatmap, binarized_mask, predicted_class = process_image_and_validate_focus(
            image_path, classification_model,
            mask_dir, predicted_dir,
            return_visuals=True  # <- Make sure your function supports this
        )

        # Save the visualization as a single PNG with 4 side-by-side panels
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))
        axs[0].imshow(original_img / 255.0)
        axs[0].set_title(f"Original\nPred: {predicted_class}")

        axs[1].imshow(gt_mask, cmap='gray')
        axs[1].set_title("Ground Truth Mask")

        axs[2].imshow(original_img / 255.0)
        axs[2].imshow(heatmap, cmap='jet', alpha=0.5)
        axs[2].set_title("Grad-CAM")

        axs[3].imshow(binarized_mask, cmap='gray')
        axs[3].set_title(f"Binary CAM\nIoU: {iou_score:.2f}")
        for ax in axs:
            ax.axis('off')

        plt.tight_layout()

        # Save to results/initial_visualizations/
        visualization_path = os.path.join(visualization_dir, f"{base_name}_viz.png")
        plt.savefig(visualization_path)
        plt.close()
        print(f"🖼️ Saved visualization: {visualization_path}")

        # Save IoU score
        iou_results.append((filename, round(iou_score, 4)))

# --- Save CSV ---
csv_output_path = os.path.join(results_dir, "iou_scores_pre_trained.csv")
with open(csv_output_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(iou_results)

# --- Print Average IoU ---
iou_values = [score for _, score in iou_results[1:]]
average_iou = sum(iou_values) / len(iou_values) if iou_values else 0
print(f"\n📄 IoU scores saved to: {csv_output_path}")
print(f"📊 Average IoU over {len(iou_values)} images: {average_iou:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_177']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


🛠️ Ground truth lesion mask not found. Generating: scab_6_masked.jpg
IoU for scab_6: 0.0125
🖼️ Saved visualization: results/initial_visualizations/scab_6_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step


🛠️ Ground truth lesion mask not found. Generating: rust_17_masked.jpg
IoU for rust_17: 0.0988
🖼️ Saved visualization: results/initial_visualizations/rust_17_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_4_masked.jpg
IoU for scab_4: 0.3102
🖼️ Saved visualization: results/initial_visualizations/scab_4_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


...

🛠️ Ground truth lesion mask not found. Generating: scab_14_masked.jpg
IoU for scab_14: 0.2589
🖼️ Saved visualization: results/initial_visualizations/scab_14_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_15_masked.jpg
IoU for scab_15: 0.0732
🖼️ Saved visualization: results/initial_visualizations/scab_15_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_12_masked.jpg
IoU for scab_12: 0.1379
🖼️ Saved visualization: results/initial_visualizations/scab_12_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_16_masked.jpg
IoU for scab_16: 0.0685
🖼️ Saved visualization: results/initial_visualizations/scab_16_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


🛠️ Ground truth lesion mask not found. Generating: healthy_16_masked.jpg
IoU for healthy_16: 0.1320
🖼️ Saved visualization: results/initial_visualizations/healthy_16_viz.png

📄 IoU scores saved to: results/iou_scores_pre_trained.csv
📊 Average IoU over 60 images: 0.0823


In [ ]:
import shutil

# Destination: move it to the current directory for download
shutil.copy(csv_output_path, "./iou_scores_pre_trained.csv")

print("✅ CSV file copied to current directory as 'iou_scores_pre_trained.csv'")

✅ CSV file copied to current directory as 'iou_scores_pre_trained.csv'


In [ ]:
from google.colab import files

files.download("iou_scores_pre_trained.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🧪 11. Fine-tuning the model

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Class mapping from filenames
class_mapping = {"healthy": 0, "rust": 1, "scab": 2}
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = len(class_mapping)
EPOCHS = 10
train_dir = "/content/apple_disease_dataset/train"
val_dir = "/content/apple_disease_dataset/val"

### 🔎 1. Create DataFrames from Filenames

In [ ]:
# If in case you want to delete folders

import shutil

shutil.rmtree("/content/original_dataset", ignore_errors=True)
shutil.rmtree("/content/original_dataset.zip", ignore_errors=True)
shutil.rmtree("/content/apple_disease_dataset", ignore_errors=True)

In [ ]:
from google.colab import files
import zipfile, os

# ✅ Step 1: Upload the zip file
print("🔼 Please upload your dataset ZIP file")
uploaded = files.upload()

# ✅ Step 2: Define extraction target
zip_file_path = list(uploaded.keys())[0]  # Name of the uploaded file
base_extract_dir = "original_dataset"
image_extract_dir = os.path.join(base_extract_dir, "images")

# ✅ Ensure target directories exist
os.makedirs(image_extract_dir, exist_ok=True)

# ✅ Step 3: Flatten and extract all image files to original_dataset/images/
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for member in zip_ref.infolist():
        filename = os.path.basename(member.filename)
        if not filename:  # Skip directories
            continue
        source = zip_ref.open(member)
        target_path = os.path.join(image_extract_dir, filename)
        with open(target_path, "wb") as target:
            target.write(source.read())

print(f"✅ All files extracted to: '{image_extract_dir}'")

# Optional: Show count of images
img_files = [f for f in os.listdir(image_extract_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"📸 Total images extracted: {len(img_files)}")

🔼 Please upload your dataset ZIP file


Saving original_dataset.zip to original_dataset.zip
✅ All files extracted to: 'original_dataset/images'
📸 Total images extracted: 300


#### In this cell, the code is for splitting the images into training and validation sets

In [ ]:
import os
import shutil
import random

# Paths
root_dir = "original_dataset"
source_dir = os.path.join(root_dir, "images")  # 300 original images
train_dir = os.path.join(root_dir, "train")
val_dir = os.path.join(root_dir, "val")

# Class mapping
class_mapping = {"healthy": 0, "rust": 1, "scab": 2}

# Create train/val directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Organize images by class
class_to_files = {cls: [] for cls in class_mapping}
for fname in os.listdir(source_dir):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        cls = fname.split('_')[0]
        if cls in class_to_files:
            class_to_files[cls].append(fname)

# Split and copy images
split_ratio = 0.8
train_count = 0
val_count = 0

for cls, files in class_to_files.items():
    random.shuffle(files)
    split_index = int(len(files) * split_ratio)
    train_files = files[:split_index]
    val_files = files[split_index:]

    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(train_dir, f))
        train_count += 1

    for f in val_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(val_dir, f))
        val_count += 1

print("✅ Dataset split completed.")
print(f"📂 Training images: {train_count}")
print(f"📂 Validation images: {val_count}")

✅ Dataset split completed.
📂 Training images: 240
📂 Validation images: 60


In [ ]:
def create_dataframe_from_filenames(directory):
    """
    Creates a pandas DataFrame from image filenames in a directory.

    Assumes each filename starts with the class label (e.g., 'rust_001.jpg').
    Extracts the label, maps it using `class_mapping`, and stores filename-label pairs.

    Args:
        directory (str): Path to the directory containing image files.

    Returns:
        pd.DataFrame: A DataFrame with 'filename' and 'label' columns.
    """
    data = []
    for fname in os.listdir(directory):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            label_str = fname.split('_')[0]
            label = class_mapping[label_str]
            data.append({"filename": fname, "label": label})
    return pd.DataFrame(data)

train_df = create_dataframe_from_filenames(train_dir)
val_df = create_dataframe_from_filenames(val_dir)

### 🧪 2. Data Generators from DataFrames

In [ ]:
# Convert labels to categorical class indices
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=20, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    directory=val_dir,
    x_col='filename',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 240 validated image filenames belonging to 3 classes.
Found 60 validated image filenames belonging to 3 classes.


### 🔧 3. Modify & Train ResNet50

In [ ]:
base_model = get_classification_model()
# Modify to exclude the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

for layer in base_model.layers[:143]:  # Freeze base
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Train ---
checkpoint_path = "finetuned_resnet50_from_filenames.h5"
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, save_best_only=True)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3405 - loss: 1.1782

15/15 ━━━━━━━━━━━━━━━━━━━━ 101s 6s/step - accuracy: 0.3416 - loss: 1.1761 - val_accuracy: 0.3333 - val_loss: 1.4128
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4681 - loss: 1.0199

15/15 ━━━━━━━━━━━━━━━━━━━━ 133s 5s/step - accuracy: 0.4675 - loss: 1.0204 - val_accuracy: 0.3333 - val_loss: 1.2982
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.4822 - loss: 1.0132

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.5933 - loss: 0.8968 - val_accuracy: 0.3333 - val_loss: 1.1715
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4862 - loss: 0.9970

15/15 ━━━━━━━━━━━━━━━━━━━━ 73s 5s/step - accuracy: 0.4897 - loss: 0.9935 - val_accuracy: 0.3667 - val_loss: 1.1611
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 72s 5s/step - accuracy: 0.5356 - loss: 0.9484 - val_accuracy: 0.3333 - val_loss: 1.2500
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 72s 5s/step - accuracy: 0.4588 - loss: 0.9593 - val_accuracy: 0.3333 - val_loss: 1.3225
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5487 - loss: 0.9423

15/15 ━━━━━━━━━━━━━━━━━━━━ 74s 5s/step - accuracy: 0.5470 - loss: 0.9434 - val_accuracy: 0.3667 - val_loss: 1.1470
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6286 - loss: 0.8866

15/15 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.6255 - loss: 0.8899 - val_accuracy: 0.3833 - val_loss: 1.0745
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6132 - loss: 0.8299

15/15 ━━━━━━━━━━━━━━━━━━━━ 72s 5s/step - accuracy: 0.6129 - loss: 0.8304 - val_accuracy: 0.5500 - val_loss: 0.9842


### ✅ 4. Load and Use Fine-Tuned Model Later

In [ ]:
from tensorflow.keras.models import load_model
classification_model = load_model("finetuned_resnet50_from_filenames.h5")

In [ ]:
from google.colab import files

# Path to your saved model file
model_path = "finetuned_resnet50_from_filenames.h5"

# Download the file
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Function for original visualizations (300 images)

In [ ]:
def process_image_and_validate_focus(image_path, classification_model, mask_dir, predicted_dir, return_visuals=False):
    """
    Processes an image, generates Grad-CAM heatmap, compares it with ground-truth lesion mask,
    calculates IoU, and optionally returns visuals or displays them inline.

    Args:
        image_path (str): Path to the input image.
        classification_model (tf.keras.Model): The trained CNN model for prediction.
        mask_dir (str): Directory containing ground-truth lesion masks.
        predicted_dir (str): Directory to save predicted Grad-CAM binary masks.
        return_visuals (bool): If True, returns image arrays and Grad-CAM results; else plots inline.

    Returns:
        tuple: (image filename, IoU score) by default;
               additionally returns visual arrays and predicted class label if return_visuals is True.
    """
    base_name, _ = os.path.splitext(os.path.basename(image_path))
    image_class = base_name.split('_')[0]

    preprocessed_img, original_img_array = load_and_preprocess_image(image_path)

    # Predict class
    predictions = classification_model.predict(preprocessed_img)
    predicted_index = np.argmax(predictions)
    # Define predicted_class_label before the if/else block
    predicted_class_label = class_mapping[predicted_index]

    # Grad-CAM heatmap
    heatmap, _ = make_gradcam_heatmap(
        preprocessed_img, classification_model,
        LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAME,
        pred_index=predicted_index
    )
    heatmap_resized = cv2.resize(heatmap, (IMG_WIDTH, IMG_HEIGHT))
    binarized_grad_cam_mask = binarize_heatmap(heatmap_resized, threshold=0.5)

    # Save predicted mask
    predicted_mask_filename = f"{base_name}_gradcam.jpg"
    predicted_mask_path = os.path.join(predicted_dir, predicted_mask_filename)
    keras_image.save_img(predicted_mask_path, np.expand_dims(binarized_grad_cam_mask * 255, axis=-1), scale=False)

    # Load or generate ground truth mask
    true_mask_filename = f"{base_name}_masked.jpg"
    true_mask_path = os.path.join(mask_dir, true_mask_filename)
    if not os.path.exists(true_mask_path):
        print(f"🛠️ Ground truth lesion mask not found. Generating: {true_mask_filename}")
        lesion_mask = generate_lesion_mask_from_image(original_img_array)
        keras_image.save_img(true_mask_path, np.expand_dims(lesion_mask * 255, axis=-1), scale=False)

    true_segmentation_mask = load_segmentation_mask(true_mask_path)
    if true_segmentation_mask.shape != (IMG_HEIGHT, IMG_WIDTH):
        true_segmentation_mask = cv2.resize(true_segmentation_mask, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)

    # IoU calculation
    iou_score = calculate_iou(binarized_grad_cam_mask, true_segmentation_mask)
    print(f"IoU for {base_name}: {iou_score:.4f}")

    # Return visuals or default inline plot
    if return_visuals:
        return (
            base_name,
            iou_score,
            original_img_array,
            true_segmentation_mask,
            heatmap_resized,
            binarized_grad_cam_mask,
            predicted_class_label
        )
    else:
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))
        axs[0].imshow(original_img_array / 255.0)
        axs[0].set_title(f"Original\nPred: {predicted_class_label}")
        axs[1].imshow(true_segmentation_mask, cmap='gray')
        axs[1].set_title("Ground Truth Mask")
        axs[2].imshow(original_img_array / 255.0)
        axs[2].imshow(heatmap_resized, cmap='jet', alpha=0.5)
        axs[2].set_title("Grad-CAM")
        axs[3].imshow(binarized_grad_cam_mask, cmap='gray')
        axs[3].set_title(f"Binary CAM\nIoU: {iou_score:.2f}")
        for ax in axs:
            ax.axis('off')
        plt.tight_layout()
        plt.show()

        return base_name, iou_score

In [ ]:
import shutil

shutil.rmtree("/content/results/original_visualizations", ignore_errors=True)

In [ ]:
import csv
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image

# --- Constants ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
LAST_CONV_LAYER_NAME = "conv5_block3_out"
CLASSIFIER_LAYER_NAME = "dense"

# --- Load model ---
classification_model = load_model("finetuned_resnet50_from_filenames.h5")

# --- Paths ---
base_dir = "original_dataset"
image_dir = os.path.join(base_dir, "images")
mask_dir = os.path.join(base_dir, "masks")
predicted_dir = os.path.join(base_dir, "predicted")
results_dir = "results"
visualization_dir = os.path.join(results_dir, "original_visualizations")

# --- Ensure folders exist ---
os.makedirs(mask_dir, exist_ok=True)
os.makedirs(predicted_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(visualization_dir, exist_ok=True)

# --- Class Mapping (from training) ---
class_mapping = {0: "healthy", 1: "rust", 2: "scab"}

# --- IoU results ---
iou_results = [("image_name", "iou_score")]

# --- Loop through images ---
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_dir, filename)

        # Process image and get visuals
        base_name, iou_score, original_img, gt_mask, heatmap_resized, binarized_gradcam, pred_class_label = process_image_and_validate_focus(
            image_path,
            classification_model,
            mask_dir,
            predicted_dir,
            return_visuals=True
        )

        # Save visualization
        fig, axs = plt.subplots(1, 4, figsize=(20, 5))
        axs[0].imshow(original_img / 255.0)
        axs[0].set_title(f"Original\nPred: {pred_class_label}")

        axs[1].imshow(gt_mask, cmap='gray')
        axs[1].set_title("Ground Truth Mask")

        axs[2].imshow(original_img / 255.0)
        axs[2].imshow(heatmap_resized, cmap='jet', alpha=0.5)
        axs[2].set_title("Grad-CAM")

        axs[3].imshow(binarized_gradcam, cmap='gray')
        axs[3].set_title(f"Binary CAM\nIoU: {iou_score:.2f}")

        for ax in axs:
            ax.axis('off')
        plt.tight_layout()

        vis_path = os.path.join(visualization_dir, f"{base_name}_viz.png")
        plt.savefig(vis_path)
        plt.close()
        print(f"🖼️ Saved visualization: {vis_path}")

        # Record score
        iou_results.append((filename, round(iou_score, 4)))

# Save CSV
csv_output_path = os.path.join(results_dir, "iou_scores_finetuned.csv")
with open(csv_output_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(iou_results)

# Print average IoU
iou_values = [score for _, score in iou_results[1:]]
avg_iou = sum(iou_values) / len(iou_values) if iou_values else 0
print(f"\n📄 IoU scores saved to: {csv_output_path}")
print(f"📊 Average IoU (Fine-Tuned Model): {avg_iou:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['input_layer_3']]
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


🛠️ Ground truth lesion mask not found. Generating: healthy_70_masked.jpg
IoU for healthy_70: 0.0872
🖼️ Saved visualization: results/original_visualizations/healthy_70_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_41_masked.jpg
IoU for scab_41: 0.0242
🖼️ Saved visualization: results/original_visualizations/scab_41_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


🛠️ Ground truth lesion mask not found. Generating: healthy_83_masked.jpg
IoU for healthy_83: 0.0325
🖼️ Saved visualization: results/original_visualizations/healthy_83_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


...

🛠️ Ground truth lesion mask not found. Generating: scab_16_masked.jpg
IoU for scab_16: 0.1219
🖼️ Saved visualization: results/original_visualizations/scab_16_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step


🛠️ Ground truth lesion mask not found. Generating: rust_46_masked.jpg
IoU for rust_46: 0.0617
🖼️ Saved visualization: results/original_visualizations/rust_46_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step


🛠️ Ground truth lesion mask not found. Generating: healthy_74_masked.jpg
IoU for healthy_74: 0.1023
🖼️ Saved visualization: results/original_visualizations/healthy_74_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


🛠️ Ground truth lesion mask not found. Generating: healthy_16_masked.jpg
IoU for healthy_16: 0.0877
🖼️ Saved visualization: results/original_visualizations/healthy_16_viz.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step


🛠️ Ground truth lesion mask not found. Generating: scab_43_masked.jpg
IoU for scab_43: 0.0511
🖼️ Saved visualization: results/original_visualizations/scab_43_viz.png

📄 IoU scores saved to: results/iou_scores_finetuned.csv
📊 Average IoU (Fine-Tuned Model): 0.0790


In [ ]:
import shutil

# Destination: move it to the current directory for download
shutil.copy(csv_output_path, "./iou_scores_finetuned.csv")

print("✅ CSV file copied to current directory as 'iou_scores_finetuned.csv'")

✅ CSV file copied to current directory as 'iou_scores_finetuned.csv'


In [ ]:
from google.colab import files

files.download("iou_scores_finetuned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Result:
### Average IoU Score using Pre-trained ResNet50 model: 0.0823
### Average IoU Score using Pre-finetuned ResNet50 model: 0.0739

## 12. Experiment with Alternative XAI Methods
### 📌 1. Grad-CAM++ and Score-CAM (XAI Alternatives)

In [ ]:
# --- Grad-CAM++ ---
def make_gradcampp_heatmap(img_array, model, last_conv_layer_name, classifier_layer_name, pred_index=None):
    """
    Generates a Grad-CAM++ heatmap for a given input image and model prediction.

    Grad-CAM++ is an enhanced version of Grad-CAM that takes into account higher-order
    gradients for better localization, especially useful in cases with multiple object instances
    or fine-grained class features.

    Args:
        img_array (np.array): Preprocessed image array with shape (1, H, W, C).
        model (tf.keras.Model): The trained CNN model.
        last_conv_layer_name (str): Name of the last convolutional layer in the model.
        classifier_layer_name (str): Name of the classifier (Dense) layer in the model.
        pred_index (int, optional): Index of the class for which to compute the heatmap.
                                    If None, uses the model's top prediction.

    Returns:
        np.array: A 2D Grad-CAM++ heatmap normalized between 0 and 1.
    """
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        output = predictions[:, pred_index]
        grads = tape1.gradient(output, conv_outputs)
        second_grads = tape2.gradient(grads, conv_outputs)

    alpha_num = grads ** 2
    alpha_denom = grads ** 2 * 2 + second_grads * conv_outputs
    alpha_denom = tf.where(alpha_denom != 0.0, alpha_denom, tf.ones_like(alpha_denom))
    alphas = alpha_num / alpha_denom
    weights = tf.reduce_sum(alphas * tf.nn.relu(grads), axis=(1, 2))

    cam = tf.reduce_sum(tf.nn.relu(weights[:, tf.newaxis, tf.newaxis, :] * conv_outputs), axis=-1)
    cam = tf.squeeze(cam)
    cam = tf.maximum(cam, 0)
    heatmap = cam / tf.math.reduce_max(cam)
    return heatmap.numpy()

In [ ]:
def make_scorecam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_name, pred_index=None):
    """
    Generates a Score-CAM heatmap for the given input image and model.

    Score-CAM creates a class activation map by linearly combining activation maps
    from the last convolutional layer, weighted by the classification score obtained
    when each activation map is applied as a mask to the input image.

    Args:
        img_array (np.array): Preprocessed input image with shape (1, H, W, C).
        model (tf.keras.Model): The trained CNN model.
        last_conv_layer_name (str): Name of the final convolutional layer.
        classifier_layer_name (str): Name of the classifier (Dense) layer.
        pred_index (int, optional): Target class index. If None, uses the top predicted class.

    Returns:
        np.array: A 2D Score-CAM heatmap normalized between 0 and 1.
    """
    # Get the Grad model
    grad_model = Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute activations
    conv_outputs, predictions = grad_model(img_array)
    conv_outputs = conv_outputs[0].numpy()  # Shape: (H, W, C)

    if pred_index is None:
        pred_index = tf.argmax(predictions[0])

    cam = np.zeros(shape=(IMG_HEIGHT, IMG_WIDTH), dtype=np.float32)

    for i in range(conv_outputs.shape[-1]):
        feature_map = conv_outputs[..., i]  # (H, W)
        upsampled = cv2.resize(feature_map, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LINEAR)

        # Normalize
        upsampled = np.maximum(upsampled, 0)
        upsampled /= np.max(upsampled) + 1e-8

        # Multiply normalized activation map to input image
        heatmap_input = img_array.copy()
        heatmap_input[0] *= upsampled[..., np.newaxis]

        score = model.predict(heatmap_input, verbose=0)[0][pred_index]

        cam += score * upsampled  # ✅ Both are (224, 224)

    heatmap = np.maximum(cam, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

### 📌 2. Automated Thresholding (Otsu or Percentile)

In [ ]:
from skimage.filters import threshold_otsu

def binarize_heatmap_auto(heatmap, method='otsu', percentile=95):
    """
    Binarizes a heatmap using an automatic thresholding method.

    Supports Otsu's thresholding or percentile-based thresholding to convert a
    continuous heatmap into a binary mask.

    Args:
        heatmap (np.array): A 2D array representing the heatmap.
        method (str): Thresholding method - 'otsu' or 'percentile'. Default is 'otsu'.
        percentile (int): Percentile value used if method is 'percentile'. Default is 95.

    Returns:
        np.array: Binary mask (uint8) where values are 0 or 1.
    """
    if method == 'otsu':
        thresh = threshold_otsu(heatmap)
    elif method == 'percentile':
        thresh = np.percentile(heatmap, percentile)
    else:
        raise ValueError("Unsupported thresholding method.")
    return (heatmap >= thresh).astype(np.uint8)

### 3. ✅ Main IoU Validation Function (Both XAI + Auto Thresholding)

In [ ]:
def process_image_with_xai_methods(image_path, model, mask_dir, predicted_dir, visualization_dir=None, return_visuals=False):
    """
    Processes a single image using Grad-CAM++ explainability, compares the attention map
    with the ground truth lesion mask, calculates IoU, and optionally saves visualization.

    Steps:
    - Preprocesses the image and makes predictions using the given model.
    - Computes Grad-CAM++ heatmap for the predicted class.
    - Binarizes the heatmap using Otsu's method.
    - Loads or generates the corresponding binary lesion mask.
    - Computes the IoU between Grad-CAM++ mask and ground truth mask.
    - Optionally saves visual comparisons.

    Args:
        image_path (str): Path to the input image.
        model (tf.keras.Model): Trained CNN model used for prediction and explanation.
        mask_dir (str): Directory containing ground-truth binary lesion masks.
        predicted_dir (str): Directory to save the predicted binary Grad-CAM++ masks.
        visualization_dir (str, optional): Directory to save overlay visualizations. If None, shows inline.
        return_visuals (bool): If True, returns image arrays and outputs for further processing.

    Returns:
        tuple: If return_visuals is False, returns (filename, IoU score).
               If True, also returns original image, ground truth mask, Grad-CAM++ heatmap, and binarized mask.
    """
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Load and preprocess
    img_array, original = load_and_preprocess_image(image_path)
    predictions = model.predict(img_array, verbose=0)
    top_index = np.argmax(predictions)

    # --- GradCAM++ ---
    heatmap_gc = make_gradcampp_heatmap(img_array, model, LAST_CONV_LAYER_NAME, CLASSIFIER_LAYER_NAME, top_index)
    heatmap_gc_resized = cv2.resize(heatmap_gc, (IMG_WIDTH, IMG_HEIGHT))
    binarized_gc = binarize_heatmap_auto(heatmap_gc_resized, method='otsu')

    # --- Ground Truth ---
    true_mask_path = os.path.join(mask_dir, f"{base_name}_masked.jpg")
    if not os.path.exists(true_mask_path):
        lesion_mask = generate_lesion_mask_from_image(original)
        keras_image.save_img(true_mask_path, np.expand_dims(lesion_mask * 255, axis=-1), scale=False)
    gt_mask = load_segmentation_mask(true_mask_path)
    gt_mask = cv2.resize(gt_mask, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)

    # --- IoU Score ---
    iou_gradcampp = calculate_iou(binarized_gc, gt_mask)
    print(f"{base_name} -> IoU GradCAM++: {iou_gradcampp:.4f}")

    # --- Save predicted mask ---
    keras_image.save_img(os.path.join(predicted_dir, f"{base_name}_gradcampp.jpg"), np.expand_dims(binarized_gc * 255, axis=-1), scale=False)

    # --- Visualization ---
    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    axs[0].imshow(original / 255.0)
    axs[0].set_title("Original")

    axs[1].imshow(gt_mask, cmap='gray')
    axs[1].set_title("Ground Truth Mask")

    axs[2].imshow(original / 255.0)
    axs[2].imshow(heatmap_gc_resized, cmap='jet', alpha=0.5)
    axs[2].set_title(f"GradCAM++\nIoU: {iou_gradcampp:.2f}")

    for ax in axs:
        ax.axis("off")
    plt.tight_layout()

    if visualization_dir is not None:
        os.makedirs(visualization_dir, exist_ok=True)
        vis_path = os.path.join(visualization_dir, f"{base_name}_xai.png")
        plt.savefig(vis_path)
        plt.close()
        print(f"🖼️ Saved visualization: {vis_path}")
    else:
        plt.show()

    if return_visuals:
        return (
            base_name,
            round(iou_gradcampp, 4),
            original,
            gt_mask,
            heatmap_gc_resized,
            binarized_gc
        )
    else:
        return base_name, round(iou_gradcampp, 4)

### 4. ✅ Execution Script

In [ ]:
import csv
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image

# --- Constants ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
LAST_CONV_LAYER_NAME = "conv5_block3_out"
CLASSIFIER_LAYER_NAME = "dense"

# --- Paths ---
original_dataset_dir = "original_dataset"
original_images_dir = os.path.join(original_dataset_dir, "images")

advanced_base = "advanced_results_dataset"
advanced_images = os.path.join(advanced_base, "images")
advanced_masks = os.path.join(advanced_base, "masks")
advanced_predicted = os.path.join(advanced_base, "predicted")
results_dir = "results"
visualization_dir = os.path.join(results_dir, "advanced_visualizations")

# --- Ensure directories exist ---
os.makedirs(advanced_images, exist_ok=True)
os.makedirs(advanced_masks, exist_ok=True)
os.makedirs(advanced_predicted, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)
os.makedirs(visualization_dir, exist_ok=True)

# --- Copy images ---
copied = 0
for fname in os.listdir(original_images_dir):
    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        shutil.copy(os.path.join(original_images_dir, fname), os.path.join(advanced_images, fname))
        copied += 1
print(f"✅ Copied {copied} images to {advanced_images}")

# --- Load model ---
print("📦 Loading model...")
classification_model = load_model("finetuned_resnet50_from_filenames.h5")
print("✅ Model loaded!")

# --- IoU results ---
iou_data = [("image_name", "gradcampp_score")]

# --- Process images ---
image_files = [f for f in os.listdir(advanced_images) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"\n🖼️ Found {len(image_files)} images to process.")

for fname in image_files:
    image_path = os.path.join(advanced_images, fname)
    try:
        base_name, iou_gc = process_image_with_xai_methods(
            image_path,
            classification_model,
            advanced_masks,
            advanced_predicted,
            visualization_dir,
            return_visuals=False
        )
        iou_data.append((fname, iou_gc))
    except Exception as e:
        print(f"❌ Failed to process {fname}: {e}")

# --- Save CSV ---
csv_path = os.path.join(results_dir, "iou_scores_gradcampp_only.csv")
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(iou_data)
print(f"\n📄 IoU results saved to: {csv_path}")

# --- Averages ---
gc_scores = [row[1] for row in iou_data[1:]]
avg_gc = sum(gc_scores) / len(gc_scores) if gc_scores else 0

print(f"📊 Average IoU - GradCAM++: {avg_gc:.4f}")
print("✅ All done!")

✅ Copied 300 images to advanced_results_dataset/images
📦 Loading model...


✅ Model loaded!

🖼️ Found 300 images to process.


healthy_70 -> IoU GradCAM++: 0.2697, Score-CAM: 0.2801
🖼️ Saved visualization: results/advanced_visualizations/healthy_70_xai.png
❌ Failed to process healthy_70.jpg: too many values to unpack (expected 2)


KeyboardInterrupt: 

In [ ]:
import shutil
import os
from google.colab import files

# List of folders to zip and download
folders = ["advanced_results_dataset", "apple_plant_dataset", "original_dataset", "results"]

for folder in folders:
    if os.path.exists(folder):
        # Create ZIP file
        zip_path = f"{folder}.zip"
        shutil.make_archive(folder, 'zip', folder)
        print(f"✅ Zipped: {zip_path}")

        # Trigger download
        files.download(zip_path)
    else:
        print(f"❌ Folder not found: {folder}")

✅ Zipped: advanced_results_dataset.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Zipped: apple_plant_dataset.zip


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Zipped: results.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>